In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 定义网络
print("===================定义网络=========================")
# 定义网络时，需要继承nn.Module，并实现它的forward方法，把网络中具有可学习参数的层放在构造函数__init__中。
# 如果某一层(如ReLU)不具有可学习的参数，则既可以放在构造函数中，也可以不放，但建议不放在其中，
# 而在forward中使用nn.functional代替。
#只要在nn.Module的子类中定义了forward函数，backward函数就会自动被实现(利用autograd)。
# 在forward 函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，
# 写法和标准的Python写法一致。

#LeNet的网络结构
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()

        # 卷积层 '1'表示输入图片为单通道, '6'表示输出通道数，'5'表示卷积核为5*5
        #步长默认是1
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 卷积层
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全连接层，y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 卷积 -> 激活 -> 池化
        #过滤器为(2, 2)步长与过滤器大小一样
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # reshape，‘-1’表示自适应
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print("LeNet网络:")
print(net)#直接打印类中的成员属性

params = list(net.parameters())#网络的可学习参数通过net.parameters()返回
print("参数个数：",len(params))
print("参数：")
for name,parameters in net.named_parameters():#net.named_parameters可同时返回可学习的参数及名称。
    print(name,':',parameters.size())

#输入
print("===================输入=========================")
#需要注意的是，torch.nn只支持mini-batches，不支持一次只输入一个样本，即一次必须是一个batch。
# 但如果只想输入一个样本，则用 input.unsqueeze(0)将batch_size设为１。
input = t.randn(1, 1, 32, 32)
output = net(input)
print("output:",output)

#损失函数
print("===================损失函数=========================")
#nn实现了神经网络中大多数的损失函数，例如nn.MSELoss用来计算均方误差，nn.CrossEntropyLoss用来计算交叉熵损失。
target = t.arange(0,10,dtype=t.float).view(1,10)#view相当于reshape
print("label:",target)
criterion = nn.MSELoss()#均方误差
loss = criterion(output, target)
print("loss:",loss)

#反向传播
print("=====================反向传播=======================")
#当调用loss.backward()时，该图会动态生成并自动微分，也即会自动计算图中参数(Parameter)的导数。
net.zero_grad() # 把net中所有可学习参数的梯度清零
print('反向传播之前 conv1.bias的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后 conv1.bias的梯度')
print(net.conv1.bias.grad)

#优化器
print("=====================优化器=======================")
# 在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，
# 例如随机梯度下降法(SGD)的更新策略如下：weight = weight - learning_rate * gradient
#手动实现如下：
learning_rate = 0.1
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)# inplace 减法
print("output:",net(input))

#torch.optim中实现了深度学习中绝大多数的优化方法，
# 例如RMSProp、Adam、SGD等，更便于使用，因此大多数时候并不需要手动写上述代码。

#新建一个优化器，指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr = 0.1)
optimizer.zero_grad()# 先梯度清零(与net.zero_grad()效果一样)
#每次反向传播都需要从新计算loss
output = net(input)
loss = criterion(output, target)
loss.backward()#反向传播
optimizer.step()#更新参数
print("output:",net(input))


===================定义网络=========================
LeNet网络:
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
参数个数： 10
参数：
conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])
===================输入=========================
output: tensor([[-0.0433,  0.0522,  0.0656, -0.0735,  0.0581,  0.0693, -0.0070,  0.0157,
         -0.0942, -0.0327]], grad_fn=<AddmmBackward>)
===================损失函数=========================
label: tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8.,